In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
def activation_fun(activ):
    return nn.ModuleDict([
        ['relu', nn.ReLU(True)],
        ['leaky_relu', nn.LeakyReLU(0.02, True)],
        ['selu', nn.SELU(True)],
        ['none', nn.Identity()]
    ])[activ]

In [3]:
def basic_block(in_c, out_c, k, depth, activation='relu', bn=False):
    blocks = []
    for _ in range(depth):
        if _ == 0: blocks.append(nn.Conv2d(in_c, out_c, kernel_size=k, padding=k//2, bias=True))
        else: blocks.append(nn.Conv2d(out_c, out_c, kernel_size=k, padding=k//2, bias=True))
            
        if bn:
            blocks.append(nn.BatchNorm2d(out_c))
        blocks.append(activation_fun(activation))
    blocks.append(nn.MaxPool2d(kernel_size=2))
    
    return blocks

In [6]:
class VGG(nn.Module):
    def __init__(self, in_c, n_cls, depths, _bn=False):
        super(VGG, self).__init__()
        
        block = basic_block
        blocks = []
        channels = [64, 128, 256, 512, 512]
        
        c = 0
        out_c = 64
        for depth in depths:
            out_c = channels[c]
            tmp_block = block(in_c, out_c, 3, depth, bn=_bn)
            in_c  = out_c
            for b in tmp_block:
                blocks.append(b)
            c+=1
            
        self.conv = nn.Sequential(
            *blocks
        )
        self.avgpool = nn.AdaptiveAvgPool2d(output_size=(7, 7))
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(7*7*512 ,4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 1000)
        )
        
    def forward(self, x):
        x = self.conv(x)
        x = self.avgpool(x)
        x = self.fc(x)
        
        return x

In [16]:
def vgg16(in_c, n_cls):
    return VGG(in_c, n_cls, [2, 2, 3, 3, 3], _bn=False)

def vgg19(in_c, n_cls):
    return VGG(in_c, n_cls, [2, 2, 4, 4, 4], _bn=False)

def vgg16_bn(in_c, n_cls):
    return VGG(in_c, n_cls, [2, 2, 3, 3, 3], _bn=True)

def vgg19_bn(in_c, n_cls):
    return VGG(in_c, n_cls, [2, 2, 4, 4, 4], _bn=True)

In [18]:
from torchsummaryM import summary

dummy = (1, 3, 224, 224)
s = summary(vgg16(3, 10), inputs=dummy)

--------------------------------------------------------------------------------------
Layer(type)               ||        Kernel Shape         Output Shape         Param #
VGG Inputs                ||                   -     [1, 3, 224, 224]               -
                          ||                                                         
01> VGG-Conv-Conv2d       ||       [3, 64, 3, 3]    [1, 64, 224, 224]           1,792
02> VGG-Conv-ReLU         ||                   -    [1, 64, 224, 224]               0
03> VGG-Conv-Conv2d       ||      [64, 64, 3, 3]    [1, 64, 224, 224]          36,928
04> VGG-Conv-ReLU         ||                   -    [1, 64, 224, 224]               0
05> VGG-Conv-MaxPool2d    ||                   -    [1, 64, 112, 112]               0
06> VGG-Conv-Conv2d       ||     [64, 128, 3, 3]   [1, 128, 112, 112]          73,856
07> VGG-Conv-ReLU         ||                   -   [1, 128, 112, 112]               0
08> VGG-Conv-Conv2d       ||    [128, 128, 3, 3]   [1

In [19]:
s = summary(vgg16_bn(3, 10), inputs=dummy)

--------------------------------------------------------------------------------------
Layer(type)               ||        Kernel Shape         Output Shape         Param #
VGG Inputs                ||                   -     [1, 3, 224, 224]               -
                          ||                                                         
01> VGG-Conv-Conv2d       ||       [3, 64, 3, 3]    [1, 64, 224, 224]           1,792
02> VGG-Conv-BatchNorm2d  ||                [64]    [1, 64, 224, 224]             128
03> VGG-Conv-ReLU         ||                   -    [1, 64, 224, 224]               0
04> VGG-Conv-Conv2d       ||      [64, 64, 3, 3]    [1, 64, 224, 224]          36,928
05> VGG-Conv-BatchNorm2d  ||                [64]    [1, 64, 224, 224]             128
06> VGG-Conv-ReLU         ||                   -    [1, 64, 224, 224]               0
07> VGG-Conv-MaxPool2d    ||                   -    [1, 64, 112, 112]               0
08> VGG-Conv-Conv2d       ||     [64, 128, 3, 3]   [1

In [20]:
s = summary(vgg19(3, 10), inputs=dummy)

--------------------------------------------------------------------------------------
Layer(type)               ||        Kernel Shape         Output Shape         Param #
VGG Inputs                ||                   -     [1, 3, 224, 224]               -
                          ||                                                         
01> VGG-Conv-Conv2d       ||       [3, 64, 3, 3]    [1, 64, 224, 224]           1,792
02> VGG-Conv-ReLU         ||                   -    [1, 64, 224, 224]               0
03> VGG-Conv-Conv2d       ||      [64, 64, 3, 3]    [1, 64, 224, 224]          36,928
04> VGG-Conv-ReLU         ||                   -    [1, 64, 224, 224]               0
05> VGG-Conv-MaxPool2d    ||                   -    [1, 64, 112, 112]               0
06> VGG-Conv-Conv2d       ||     [64, 128, 3, 3]   [1, 128, 112, 112]          73,856
07> VGG-Conv-ReLU         ||                   -   [1, 128, 112, 112]               0
08> VGG-Conv-Conv2d       ||    [128, 128, 3, 3]   [1

In [21]:
s = summary(vgg19_bn(3, 10), inputs=dummy)

--------------------------------------------------------------------------------------
Layer(type)               ||        Kernel Shape         Output Shape         Param #
VGG Inputs                ||                   -     [1, 3, 224, 224]               -
                          ||                                                         
01> VGG-Conv-Conv2d       ||       [3, 64, 3, 3]    [1, 64, 224, 224]           1,792
02> VGG-Conv-BatchNorm2d  ||                [64]    [1, 64, 224, 224]             128
03> VGG-Conv-ReLU         ||                   -    [1, 64, 224, 224]               0
04> VGG-Conv-Conv2d       ||      [64, 64, 3, 3]    [1, 64, 224, 224]          36,928
05> VGG-Conv-BatchNorm2d  ||                [64]    [1, 64, 224, 224]             128
06> VGG-Conv-ReLU         ||                   -    [1, 64, 224, 224]               0
07> VGG-Conv-MaxPool2d    ||                   -    [1, 64, 112, 112]               0
08> VGG-Conv-Conv2d       ||     [64, 128, 3, 3]   [1